<a href="https://colab.research.google.com/github/gahongayirebelise/NLP-FELLOWSHIP/blob/main/Belise_hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **SCRAPPING JOBINRWANDA**

In [ ]:
from bs4 import BeautifulSoup # Extract information from HTML Pages

In [ ]:
import bs4
import requests #Fetch HTML Pages from the Web
import pandas as pd #organise our data into tabular/tables
import re
import json

In [ ]:
content = requests.get('https://www.jobinrwanda.com/jobs/all').content #Jobs
Soup = BeautifulSoup(content, 'html.parser')
web_infos = Soup.find_all('div', class_='card-body p-2')



In [ ]:
len(web_infos)

76

In [ ]:
web_infos[0]

<div class="card-body p-2">
<a href="/job/operations-manager-6"><h5 class="card-title"><span class="field field--name-title field--type-string field--label-hidden">Operations Manager</span>
</h5></a>
<p class="card-text">
<a href="/employer/gardaworld">GardaWorld </a>  |
              Published on 03-11-2022 |
              Deadline <time class="datetime" datetime="2022-11-14T21:59:00Z">14-11-2022</time>
<br/>
                              Senior (5+ years of experience)<br/>
<span class="badge badge-primary">Job</span>
</p>
</div>

In [ ]:
web_infos[0].find('p').find('br')

<br/>

In [ ]:
web_infos[0].find('span').getText() #Title of Each Job advert
web_infos[0].find('a')['href']  #Link to the content of each job title
web_infos[0].find('p').find('a') #Link to the Institution that has posted the advert
web_infos[0].find('p').find('a').getText() #This is the name of the company

'GardaWorld '

In [ ]:
df = pd.DataFrame()
df['link'] = None
df['title'] = None
df['company_link'] = None
df['company'] = None
df['type'] = None

In [ ]:
def parse_content(content, web_info_type):
  link = []
  title = []
  company = []
  company_link = []
  job_type = []
  for index, web_info in enumerate(content):
    link.append(web_info.find('a')['href'])
    title.append(web_info.find('span').getText()) #GEt the title from
    company.append(str(web_info.find('p').find('a').getText()))
    company_link.append(web_info.find('p').find('a'))
    job_type.append(web_info_type) #Whether its a job/consultacy/tender
  return link, title, company, company_link, job_type


In [ ]:
#Jobs web_info
content_bs = BeautifulSoup(content, 'html.parser')
content_titles = content_bs.find_all('div', class_='card-body p-2')
df['link'], df['title'], df['company'], df['company_link'], df['type']= parse_content(content_titles, web_info_type='Job')

In [ ]:
df['link']

0                            /job/operations-manager-6
1                          /job/account-manager-credit
2                                   /job/legal-manager
3     /job/district-coordinatorjunior-district-manager
4                          /job/radio-sales-executives
                            ...                       
71                               /job/rwanda-tree-lead
72      /index.php/job/rwanda-potato-seed-venture-lead
73                   /job/it-operations-senior-manager
74             /job/rwanda-seed-innovation-centre-lead
75                        /job/senior-business-analyst
Name: link, Length: 76, dtype: object

In [ ]:
list_description = []
for i in range(len(df)):
  link = 'https://www.jobinrwanda.com' + df['link'][i]
  print(link, i)

https://www.jobinrwanda.com/job/operations-manager-6 0
https://www.jobinrwanda.com/job/account-manager-credit 1
https://www.jobinrwanda.com/job/legal-manager 2
https://www.jobinrwanda.com/job/district-coordinatorjunior-district-manager 3
https://www.jobinrwanda.com/job/radio-sales-executives 4
https://www.jobinrwanda.com/job/radio-presenter-0 5
https://www.jobinrwanda.com/job/driver-16 6
https://www.jobinrwanda.com/job/procurement-and-logistics-officer 7
https://www.jobinrwanda.com/job/product-manager-1 8
https://www.jobinrwanda.com/job/aftercare-vehicle-maintenance-manager 9
https://www.jobinrwanda.com/job/finance-officer-9 10
https://www.jobinrwanda.com/job/communications-officer-8 11
https://www.jobinrwanda.com/job/lecturer-communication-and-writing 12
https://www.jobinrwanda.com/job/research-associate-ra-0 13
https://www.jobinrwanda.com/job/office-assistant-0 14
https://www.jobinrwanda.com/job/head-field-officehuye 15
https://www.jobinrwanda.com/job/human-resources-assistant-0 16
h

In [ ]:
job_description_raw_content = requests.get('https://www.jobinrwanda.com/job/global-remote-sensing-manager').content

In [ ]:
job_description_raw_content

In [ ]:
job_description_bs = BeautifulSoup(job_description_raw_content, 'html.parser')

In [ ]:
job_description_bs.find_all('div', class_='employer-description') #Employer Description

In [ ]:
describe = job_description_bs.find_all('div', class_="clearfix text-formatted field field--name-field-job-full-description field--type-text-long field--label-hidden field__item") #Employer Description

In [ ]:
df['description'] = None

In [ ]:
def get_description(job_link):
  job_description_raw_content = requests.get(f'https://www.jobinrwanda.com/{job_link}').content #request raw content
  job_description_bs = BeautifulSoup(job_description_raw_content, 'html.parser')  #pass the html raw content to beautifull soup
  job_description_bs.find_all('div', class_='employer-description') #Employer Description
  #Get the entire job description
  describe = job_description_bs.find_all('div', class_="clearfix text-formatted field field--name-field-job-full-description field--type-text-long field--label-hidden field__item") #Employer Description
  description = '' #Construct an empty string to hold the paragraph
  for paragraph in describe:
    description += paragraph.getText() #Adding each paragraph we find in the job description.
  return description

In [ ]:
for position, link in enumerate(df['link']):
  df.loc[position]['description'] = get_description(job_link=link)

In [ ]:
df['company_link'][0].getText()
df['company_link'] = df[['company_link']].applymap(lambda x: x.getText())

In [ ]:
df['description'] = df[['description']].applymap(lambda x: re.compile('\n|\\xa0').sub("", x))

In [ ]:
df.head()

,link,title,company_link,company,type,description
0,/job/operations-manager-6,Operations Manager,GardaWorld,GardaWorld,Job,Job Description– Operations ManagerPosition/Em...
1,/job/account-manager-credit,Account Manager- Credit,Yellow,Yellow,Job,Position:Account Manager- CreditLocations:Mala...
2,/job/legal-manager,Legal Manager,COPEDU PLC,COPEDU PLC,Job,"NOTICE OF RECRUITMENTCOPEDU PLC, is a trading ..."
3,/job/district-coordinatorjunior-district-manager,District Coordinator/Junior District Manager,EarthEnable Rwanda,EarthEnable Rwanda,Job,JOB DESCRIPTION:DISTRICT COORDINATOR/JUNIOR DI...
4,/job/radio-sales-executives,Radio Sales Executives,ROYAL RADIO LTD,ROYAL RADIO LTD,Job,JOB OPPORTUNITY94.3 ROYAL FM is opening up a j...


In [ ]:
tags = "#informationtechnology #technology #it #cybersecurity #tech #computerscience #programming #business #coding #innovation #software #python #information #computer #informationsecurity #security #technologynews #java #networking #hacking #programmer #linux #technologyrocks #coder #technologythesedays #cloudcomputing #education #engineering #itservices #newtechnology"

In [ ]:
tags.split('#')

['',
 'informationtechnology ',
 'technology ',
 'it ',
 'cybersecurity ',
 'tech ',
 'computerscience ',
 'programming ',
 'business ',
 'coding ',
 'innovation ',
 'software ',
 'python ',
 'information ',
 'computer ',
 'informationsecurity ',
 'security ',
 'technologynews ',
 'java ',
 'networking ',
 'hacking ',
 'programmer ',
 'linux ',
 'technologyrocks ',
 'coder ',
 'technologythesedays ',
 'cloudcomputing ',
 'education ',
 'engineering ',
 'itservices ',
 'newtechnology']

# ***Using Regular expression and key-words to retrieve IT/Software related job/consultancy opportunities(Tenders).***

In [ ]:
keywords = """information|technology|IT|cybersecurity|tech|computer science|programming|coding|software|python|computer|information security|technologynews|java|networking|hacking|programmer|linux|coder|cloudcomputing|computer engineering"""

In [ ]:
df['IT Job'] = False
df['keywords found'] = None
def isIT_Job():
  pattern = re.compile(keywords)
  for index in range(len(df)):
    try:
      if len(pattern.findall(df.iloc[index]["title"])) >= 1:
        df['IT Job'][index] = True
        df['keywords found'][index] = set(pattern.findall(df.iloc[index]["title"]))
        # print(set(pattern.findall(df.iloc[index]["Job Description"])), df.iloc[index]["Job Title"])
    except TypeError:
      pass
      # print("content: ", df.iloc[index]["Job Description"])
    #
isIT_Job()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
df3 = df[df['IT Job'] == True]
df3

,link,title,company_link,company,type,description,IT Job,keywords found
41,/job/it-and-mis-director,IT and MIS Director,CHANCEN International Rwanda,CHANCEN International Rwanda,Job,CHANCEN International is a non-profit organiza...,True,{IT}
73,/job/it-operations-senior-manager,IT Operations Senior Manager,One Acre Fund,One Acre Fund,Job,"About One Acre FundFounded in 2006, One Acre F...",True,{IT}


In [ ]:
type(df3[['description']])

pandas.core.frame.DataFrame

In [ ]:
df3.to_dict("r")

# HOSTING **WORK**

In [ ]:
!pip install -U easynmt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 28.3 MB/s 
     |████████████████████████████████| 1.3 MB 38.9 MB/s 
     |████████████████████████████████| 68 kB 6.4 MB/s 
     |████████████████████████████████| 163 kB 42.2 MB/s 
     |████████████████████████████████| 7.6 MB 37.8 MB/s 
  Using cached pybind11-2.10.1-py3-none-any.whl (216 kB)
  Created wheel for easynmt: filename=EasyNMT-2.0.2-py3-none-any.whl size=19919 sha256=3e9c445cf87c49166c6b25abdea55a8ec399c1eba16ad577a1afdb79e48a457a
  Stored in directory: /root/.cache/pip/wheels/d1/57/06/53ca38645e14d4537a41e5a36da2026e10f54ae88240dd5190
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3164380 sha256=5438375166ab2223a4222c140edb1bcf07ad94b0d632ebecbb85f53b10eed99e
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built easy

In [ ]:
#FastAPI
!pip install fastapi pyngrok  uvicorn nest-asyncio
# Flask
#!pip install flask pyngrok  gunicorn nest-asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 745 kB 49.4 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 80 kB 10.0 MB/s 
     |████████████████████████████████| 58 kB 6.1 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=60696bb2f3a63ff95f5320b2888ff35f3d1ba860ede44f3dbff002abe1646c6b
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
!ngrok authtoken 2HJPxGxCEP4kvWWdNAnxrxYcaO7_5ZoRfnsdZZ6BjAKBpzPLA

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:

from fastapi import FastAPI, Request
from datetime import datetime
from fastapi.responses import HTMLResponse
from easynmt import EasyNMT
from fastapi import Response

model = EasyNMT('opus-mt')#starting the model Instance

storage = FastAPI(title='MY FASTAPI') #Starting the FastAPI Instance

@storage.get('/')
def index():
    return "My name is Gahongayire Belise, This is my API"


@storage.get('/data')
def show_data():
  return df3.to_dict("r")

@storage.get('/translation')
def translate(lang: str=''):
  previous_lang = 'en'
  for row in range(len(df3)):
    try:
      df3.loc[row]['description'] = model.translate(df3.loc[row]['description'] , target_lang=lang)
      df3.loc[row]['title'] = model.translate(df3.loc[row]['title']   ,target_lang=lang)
    except AttributeError:
      message = f"""<html>
                    <h1>{lang} is not among the languages supported by easyNMT.<br>Try another language</h1></html>
                """
    return HTMLResponse(message)



11.9kB [00:00, 7.46MB/s]                   


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print("REST API started")
print("Your public API URL:", ngrok_tunnel.public_url)
print("You can for example open the following URL in your browser: {}?target_lang=en&text=Hallo%20Welt".format(ngrok_tunnel.public_url))

nest_asyncio.apply()
uvicorn.run(storage, port=8000)

INFO:     Started server process [75]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


REST API started
Your public API URL: http://4ff7-35-245-16-71.ngrok.io
You can for example open the following URL in your browser: http://4ff7-35-245-16-71.ngrok.io?target_lang=en&text=Hallo%20Welt


In [ ]:
!pkill ngrok